### **🧠 머신러닝 실습 - 이진 분류**
#### Q1. 환자의 당뇨병 여부를 예측하시오.
- 제공된 데이터 : `diabetes_train.csv`, `diabetes_test.csv`
- 예측할 컬럼 : `Outcome` (0: 정상, 1: 당뇨병)


학습용 데이터(train)을 이용해 환자의 당뇨병을 예측하는 모델을 만든 후 이를 평가용 데이터(test)에 적용해 얻은 예측값을 다음과 같은 형식의 csv 파일로 생성하시오.


제출 파일은 다음 1개의 컬럼을 포함해야 한다.
- pred : 예측값 (당뇨병일 확률)
- 제출 파일명 : `new.csv`


제출한 모델의 성능은 ROC-AUC 평가지표에 따라 채점한다.

In [43]:
import pandas as pd

## 데이터 불러오기
train = pd.read_csv('diabetes_train.csv')
test = pd.read_csv('diabetes_test.csv')

## EDA
# print(train.shape, test.shape) # ((614, 9), (154, 8))
# train.info() # not object
# test.info()
# train.describe()
# print(train.isnull().sum())
# print(test.isnull().sum())
# print(train['Outcome'].value_counts()) # 0: 403 / 1: 211

## 데이터 전처리
y_train = train.pop('Outcome')
## - 스케일링
from sklearn.preprocessing import MinMaxScaler

minmax = MinMaxScaler()
train = minmax.fit_transform(train)
test = minmax.transform(test)


## 검증 데이터 나누기
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(train, y_train, test_size=0.2, random_state=42)
# print(X_train.shape, X_val.shape, y_train.shape, y_val.shape) # (491, 8) (123, 8) (491,) (123,)

## 모델 학습 및 평가
from sklearn.ensemble import RandomForestClassifier

# 하이퍼파라미터 튜닝
rf = RandomForestClassifier(max_depth=5, n_estimators=500, random_state=0)
# rf = RandomForestClassifier(random_state=0) # 기본

rf.fit(X_train, y_train)
pred = rf.predict_proba(X_val)
# print(rf.classes_)
# print(pred)

from sklearn.metrics import roc_auc_score

roc_auc = roc_auc_score(y_val, pred[:,1])
print('roc_auc:', roc_auc)

## 예측 및 결과 파일 생성
pred = rf.predict_proba(test)
result = pd.DataFrame({'pred':pred[:,1]})
result.to_csv('new.csv', index=False)

pd.read_csv('new.csv').shape


roc_auc: 0.8234969663541093


(154, 1)

스케일링 전 roc_auc : 0.8167402095973524



스케일링 후 roc_auc : 0.8234969663541093


**[하이퍼파라미터 튜닝]**
- `max_depth` : 트리의 최대 깊이, 과적합 방지, 3~7 정도 적당
- `n_estimators` : 트리의 개수, 높을수록 안정적 예측 가능(학습 시간 소요), 200~500 정도 적당

---

#### Q2. 새로운 일자리를 찾을지 예측하시오.
- 제공된 데이터 : `hr_train.csv`, `hr_test.csv`
- 예측할 컬럼 : `target` (0: 새로운 일자리를 찾지 않음, 1: 새로운 일자리를 찾음)


학습용 데이터(train)을 이용해 새 일자리를 찾을지 예측하는 모델을 만든 후 이를 평가용 데이터(test)에 적용해 얻은 예측값을 다음과 같은 형식의 csv 파일로 생성하시오.


제출 파일은 다음 1개의 컬럼을 포함해야 한다.
- pred : 예측값 (이직할 확률)
- 제출 파일명 : `new2.csv`


제출한 모델의 성능은 ROC-AUC 평가지표에 따라 채점한다.

In [52]:
## 라이브러리 및 데이터 불러오기
import pandas as pd

train = pd.read_csv('hr_train.csv')
test = pd.read_csv('hr_test.csv')

## EDA
# print(train.info())
# print(test.info())
# print(train.isnull().sum()) # gender, enrolled_university, education_level, major_discipline, experience, company_size, company_type, last_new_job
# print(test.isnull().sum())
# train.describe()
# train.describe(include='O')
# test.describe(include='O')
# train['target'].value_counts() # 0.0: 11517 / 1.0: 3809
# print(train.shape, test.shape) # (15326, 14) (3832, 13)

## 데이터 전처리
# train = train.drop('enrollee_id', axis=1)
# test = test.drop('enrollee_id', axis=1)

train = train.fillna('X')
test = test.fillna('X')

# print(train.isnull().sum().sum(), test.isnull().sum().sum())

y_train = train.pop('target')

# 원-핫 인코딩
train = pd.get_dummies(train)
test = pd.get_dummies(test)
# train.shape, test.shape # 불일치

df = pd.concat([train, test], axis=0)
df_oh = pd.get_dummies(df)
train = df_oh.iloc[:len(train)].copy()
test = df_oh.iloc[len(train):].copy()

# train.shape, test.shape # 일치

# 라벨 인코딩
from sklearn.preprocessing import LabelEncoder

cols_obj = train.select_dtypes(include='O').columns

for col in cols_obj :
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])

train = df[:len(train)].copy()
test = df[len(train):].copy()

# 스케일링
from sklearn.preprocessing import MinMaxScaler

cols_num = train.select_dtypes(exclude='O').columns

minmax = MinMaxScaler()
train[cols_num] = minmax.fit_transform(train[cols_num])
test[cols_num] = minmax.transform(test[cols_num])

## 검증 데이터 나누기
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(train,
                                                  y_train,
                                                  test_size=0.2, 
                                                  random_state=42)

## 모델 학습 및 평가
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(random_state=0, max_depth=5, n_estimators=500)
rf.fit(X_train, y_train)
y_pred = rf.predict_proba(X_val)

# print(rf.classes_)
# print(y_pred)

from sklearn.metrics import roc_auc_score

roc_auc = roc_auc_score(y_val, y_pred[:,1])
print('roc-auc:', roc_auc)

## 예측 및 결과 파일 생성
pred = rf.predict_proba(test)
result = pd.DataFrame({'pred':pred[:,1]})
result.to_csv('new2.csv', index=False)

pd.read_csv('new2.csv').head()




roc-auc: 0.7952435945564116


,pred
0,0.234171
1,0.394253
2,0.411359
3,0.140237
4,0.157046


In [ ]:
cols = train.select_dtypes(include='O')

for col in cols:
    train_set = set(train[col])
    test_set = set(test[col])

    if train_set == test_set :
        print(col, '동일')
    else :
        print(col, '동일하지 않음')

---
#### Q3. 신용카드 신청자의 채무 불이행을 예측하시오.
- 제공된 데이터 : `creditcard_train.csv`, `creditcard_test.csv`
- 예측할 컬럼 : `STATUS` (0: 채무 이행, 1: 채무 불이행)


학습용 데이터(train)을 이용해 미래의 채무 불이행을 예측하는 모델을 만든 후 이를 평가용 데이터(test)에 적용해 얻은 예측값을 다음과 같은 형식의 csv 파일로 생성하시오.


제출 파일은 다음 1개의 컬럼을 포함해야 한다.
- pred : 예측값 (채무 불이행 확률)
- 제출 파일명 : `new3.csv`


제출한 모델의 성능은 F1 평가지표에 따라 채점한다.

In [83]:
import pandas as pd

train = pd.read_csv('creditcard_train.csv')
test = pd.read_csv('creditcard_test.csv')

cols_obj = train.select_dtypes(include='O').columns

for col in cols_obj :
    train_set = set(train[col])
    test_set = set(test[col])

    if train_set == test_set :
        print(col, '동일')
    else :
        print(col, '동일하지 않음')

print('-----------------------')

## EDA
# train.info()
# test.info()
# train.shape, test.shape # ((25519, 19), (7591, 18))
# train.isnull().sum() # OCCUPATION_TYPE 
# test.isnull().sum()
# train['STATUS'].value_counts() # 0: 25085 / 1: 434

## 데이터 전처리
# train['OCCUPATION_TYPE'].value_counts()
train['OCCUPATION_TYPE'] = train['OCCUPATION_TYPE'].fillna(train['OCCUPATION_TYPE'].mode()[0])

train = train.drop('ID', axis=1)
test = test.drop('ID', axis=1)

cols_obj = train.select_dtypes(include='O').columns

y_train = train.pop('STATUS')

# # 원-핫 인코딩
# train = pd.get_dummies(train)
# test = pd.get_dummies(test)
# # train.shape, test.shape

# 라벨 인코딩
from sklearn.preprocessing import LabelEncoder

for col in cols_obj :
    le = LabelEncoder()
    train[col] = le.fit_transform(train[col])
    test[col] = le.transform(test[col])
    
# # 스케일링 (개선 x)
# from sklearn.preprocessing import MinMaxScaler

# cols_num = train.select_dtypes(exclude='O').columns

# minmax = MinMaxScaler()
# train[cols_num] = minmax.fit_transform(train[cols_num])
# test[cols_num] = minmax.transform(test[cols_num])

## 검증 데이터 분할
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(train, 
                                                  y_train, 
                                                  test_size=0.2, 
                                                  random_state=0)

## 모델 학습 및 평가
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(random_state=0, n_estimators=500, class_weight='balanced')
rf.fit(X_train, y_train)
y_pred = rf.predict(X_val)
print(rf.classes_)
print(y_pred)

from sklearn.metrics import f1_score

f1 = f1_score(y_val, y_pred)
print('f1-score:', f1)

## 예측 및 결과 파일 생성
pred = rf.predict(test)
result = pd.DataFrame({'pred':pred})
result.to_csv('new3.csv', index=False)

pd.read_csv('new3.csv').shape


CODE_GENDER 동일
FLAG_OWN_CAR 동일
FLAG_OWN_REALTY 동일
NAME_INCOME_TYPE 동일
NAME_EDUCATION_TYPE 동일
NAME_FAMILY_STATUS 동일
NAME_HOUSING_TYPE 동일
OCCUPATION_TYPE 동일하지 않음
-----------------------
[0 1]
[0 0 0 ... 0 0 0]
f1-score: 0.3241106719367589


(7591, 1)

---
### **🧠 머신러닝 실습 - 다중 분류**
#### Q1. 은행 정보로 신용 등급을 예측하시오.
- 제공된 데이터 : `score_train.csv`, `score_test.csv`
- 예측할 컬럼 : `Credit_Score` (Good, Standard, Poor)


학습용 데이터(train)을 이용해 신용 등급을 예측하는 모델을 만든 후 이를 평가용 데이터(test)에 적용해 얻은 예측값을 다음과 같은 형식의 csv 파일로 생성하시오.


제출 파일은 다음 1개의 컬럼을 포함해야 한다.
- pred : 예측값
- 제출 파일명 : `new4.csv`


제출한 모델의 성능은 F1-macro 평가지표에 따라 채점한다.

In [55]:
import pandas as pd

train = pd.read_csv('score_train.csv')
test = pd.read_csv('score_test.csv')

## EDA
# train.head()
# train.info()
# test.info()
# train.isnull().sum()
# test.isnull().sum()
# print(train.shape, test.shape)
# train['Credit_Score'].value_counts()

## 데이터 전처리
y_train = train.pop('Credit_Score')
cols_obj = train.select_dtypes(include='O').columns

for col in cols_obj :
    train_set = set(train[cols_obj])
    test_set = set(test[cols_obj])
    
    if train_set == test_set :
        print(col, '동일')
    else :
        print(col, '동일하지 않음')
print('---------------------')

# 원-핫 인코딩
train = pd.get_dummies(train)
test = pd.get_dummies(test)
train.shape, test.shape

# 스케일링
from sklearn.preprocessing import MinMaxScaler

cols_num = train.select_dtypes(exclude='O').columns

minmax = MinMaxScaler()
train[cols_num] = minmax.fit_transform(train[cols_num])
test[cols_num] = minmax.transform(test[cols_num])

## 검증 데이터 나누기
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(train, 
                                                  y_train, 
                                                  test_size=0.2, 
                                                  random_state=0)
X_train.shape, X_val.shape, y_train.shape, y_val.shape

## 모델 학습 및 평가
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(random_state=0)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_val)
print(rf.classes_)
print('---------------------')

from sklearn.metrics import f1_score

f1 = f1_score(y_val, y_pred, average='macro')
print('f1-macro:', f1)

## 예측 및 결과 파일 생성
pred = rf.predict(test)
result = pd.DataFrame({'pred':pred})
result.to_csv('new4.csv', index=False)

pd.read_csv('new4.csv').shape

Payment_of_Min_Amount 동일
Credit_Mix 동일
Payment_Behaviour 동일
---------------------
['Good' 'Poor' 'Standard']
---------------------
f1-macro: 0.7006138262986147


(1499, 1)

#### Q2. 주어진 데이터에서 약물의 종류를 예측하시오.
- 제공된 데이터 : `drug_train.csv`, `drug_test.csv`
- 예측할 컬럼 : `Drug` (DrugY, DrugX, drugA, drugC, drugB)


학습용 데이터(train)을 이용해 약물의 종류를 예측하는 모델을 만든 후 이를 평가용 데이터(test)에 적용해 얻은 예측값을 다음과 같은 형식의 csv 파일로 생성하시오.


제출 파일은 다음 1개의 컬럼을 포함해야 한다.
- pred : 예측값
- 제출 파일명 : `new5.csv`


제출한 모델의 성능은 f1-macro 평가지표에 따라 채점한다.

In [79]:
import pandas as pd

train = pd.read_csv('drug_train.csv')
test = pd.read_csv('drug_test.csv')

## EDA
train.head()
train.shape, test.shape # ((100, 6), (100, 5))
train.isnull().sum()
test.isnull().sum()
train.describe()
train.describe(include='O')

y_train = train.pop('Drug')

cols_obj = train.select_dtypes(include='O').columns
cols_num = train.select_dtypes(exclude='O').columns

for col in cols_obj :
    train_set = set(train[col])
    test_set = set(test[col])
    if train_set == test_set :
        print(col, '동일')
    else :
        print(col, '동일하지 않음')

print('---------------------------')

## 데이터 전처리
# 원-핫 인코딩
train = pd.get_dummies(train)
test = pd.get_dummies(test)
train.shape, test.shape

# 라벨 인코딩
# from sklearn.preprocessing import LabelEncoder

# for col in cols_obj : 
#     le = LabelEncoder()
#     train[col] = le.fit_transform(train[col])
#     test[col] = le.transform(test[col])

## 검증 데이터 나누기
from sklearn.model_selection import train_test_split

# X_train, X_val, y_train, y_val = train_test_split(train, y_train, test_size=0.2, random_state=0)

## 모델 학습 및 평가
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
## 크로스 밸리데이션 활용
from sklearn.model_selection import cross_val_score

rf = RandomForestClassifier(random_state=0)
# rf.fit(X_train, y_train)
# y_pred = rf.predict(X_val)
# print(rf.classes_)
# print(y_pred)
print('---------------------------')

f1 = cross_val_score(rf, train, y_train, cv=3, scoring='f1_macro')
print('f1:', f1.mean())

## 예측 및 결과 파일 생성
rf.fit(train, y_train)
pred = rf.predict(test)
result = pd.DataFrame({'pred':pred})
result.to_csv('new5.csv', index=False)

pd.read_csv('new5.csv').shape

Sex 동일
BP 동일
Cholesterol 동일
---------------------------
---------------------------
f1: 0.9074643874643874


(100, 1)

#### Q3. 주어진 데이터에서 유리의 종류를 예측하시오.
- 제공된 데이터 : `glass_train.csv`, `glass_test.csv`
- 예측할 컬럼 : `Type` (1, 2, 3, 4, 5, 6, 7)


학습용 데이터(train)을 이용해 유리의 종류를 예측하는 모델을 만든 후 이를 평가용 데이터(test)에 적용해 얻은 예측값을 다음과 같은 형식의 csv 파일로 생성하시오.


제출 파일은 다음 1개의 컬럼을 포함해야 한다.
- pred : 예측값
- 제출 파일명 : `new6.csv`


제출한 모델의 성능은 f1-weighted 평가지표에 따라 채점한다.

In [4]:
import pandas as pd

train = pd.read_csv('glass_train.csv')
test = pd.read_csv('glass_test.csv')

## EDA
# train.shape, test.shape # ((149, 10), (65, 9))
# train.info()
# test.info()
# train.isnull().sum()  
# test.isnull().sum()
# train['Type'].value_counts()
# train.describe()

## 데이터 전처리
y_train = train.pop('Type')

# 원-핫 인코딩
train = pd.get_dummies(train)
test = pd.get_dummies(test)
# train.shape, test.shape

# 라벨 인코딩
# from sklearn.preprocessing import LabelEncoder

# df = pd.concat([train, test])

# cols = train.columns
# for col in cols :
#     le = LabelEncoder()
#     df[col] = le.fit_transform(df[col])

# train = df.iloc[:len(train)]
# test = df.iloc[len(train):]


## 검증 데이터 나누기
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(train, y_train, test_size=0.2, random_state=0)

## 모델 학습 및 평가
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

rf = RandomForestClassifier(random_state=0, max_depth=5, n_estimators=700)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_val)
# print(rf.classes_)
# print(y_pred)

f1 = f1_score(y_val, y_pred, average='weighted')
print('f1:', f1)

## 예측 및 결과 파일 생성하기
pred = rf.predict(test)
result = pd.DataFrame({'pred':pred})
result.to_csv('new6.csv', index=False)

pd.read_csv('new6.csv').shape



f1: 0.6507936507936507


(65, 1)

---
### **🧠 머신러닝 실습 - 회귀**
#### Q1. 항공권 티켓 가격을 예측하시오.
- 제공된 데이터 : `flight_train.csv`, `flight_test.csv`
- 예측할 컬럼 : `price`


학습용 데이터(train)을 이용해 티켓 가격을 예측하는 모델을 만든 후 이를 평가용 데이터(test)에 적용해 얻은 예측값을 다음과 같은 형식의 csv 파일로 생성하시오.


제출 파일은 다음 1개의 컬럼을 포함해야 한다.
- pred : 예측값 (가격)
- 제출 파일명 : `new7.csv`


제출한 모델의 성능은 RMSE 평가지표에 따라 채점한다.

In [65]:
import pandas as pd

train = pd.read_csv('flight_train.csv')
test = pd.read_csv('flight_test.csv')

## EDA
# train.info()
# test.info()
# print(train.shape, test.shape) # (10505, 11) (4502, 10)
# train.describe()
# train.describe(include='O')
# train.isnull().sum()
# test.isnull().sum()

print ('------ 카테고리 동일 여부 ------')
cols_obj = train.select_dtypes(include='O').columns
for col in cols_obj :
    train_set = set(train[col])
    test_set = set(test[col])
    if train_set == test_set :
        print(col, '동일')
    else :
        print(col, '동일하지 않음')
        
## 데이터 전처리
y_train = train.pop('price')

train['new_flight'] = train['flight'].str.split('-').str[1].astype(int)
test['new_flight'] = test['flight'].str.split('-').str[1].astype(int)

train = train.drop('flight', axis=1)
test = test.drop('flight', axis=1)

# 원-핫 인코딩
train = pd.get_dummies(train)
test = pd.get_dummies(test)
# print(train.shape, train_oh.shape, test.shape, test_oh.shape)

# df = pd.concat([train, test], axis=0)
# df = pd.get_dummies(df)
# train = df[:len(train)]
# test = df[len(train):]
# train.shape, test.shape

# 스케일링
from sklearn.preprocessing import StandardScaler

cols_num = train.select_dtypes(exclude='O').columns
rb = StandardScaler()
train[cols_num] = rb.fit_transform(train[cols_num])
test[cols_num] = rb.transform(test[cols_num])


## 검증 데이터 나누기
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(train, y_train, test_size=0.2, random_state=0)
X_train.shape, X_val.shape, y_train.shape, y_val.shape

## 모델 학습 및 평가
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import root_mean_squared_error

rf = RandomForestRegressor(random_state=0, max_depth=20, n_estimators=200)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_val)

rmse = root_mean_squared_error(y_val, y_pred)
print ('\n------ RMSE SCORE ------')
print('rmse:', rmse)  

## 예측 및 결과 파일 생성
pred = rf.predict(test)
result = pd.DataFrame({'pred':pred}).to_csv('new7.csv', index=False)

print ('\n------ 최종 결과 파일 확인 ------')
pd.read_csv('new7.csv').shape



------ 카테고리 동일 여부 ------
airline 동일
flight 동일하지 않음
source_city 동일
departure_time 동일
stops 동일
arrival_time 동일
destination_city 동일
class 동일

------ RMSE SCORE ------
rmse: 3674.8958942154045

------ 최종 결과 파일 확인 ------


(4502, 1)

#### Q2. 노트북 가격을 예측하시오.
- 제공된 데이터 : `laptop_train.csv`, `laptop_test.csv`
- 예측할 컬럼 : `Price`


학습용 데이터(train)을 이용해 노트북 가격을 예측하는 모델을 만든 후 이를 평가용 데이터(test)에 적용해 얻은 예측값을 다음과 같은 형식의 csv 파일로 생성하시오.


제출 파일은 다음 1개의 컬럼을 포함해야 한다.
- pred : 예측값 (가격)
- 제출 파일명 : `new8.csv`


제출한 모델의 성능은 R²(결정계수) 평가지표에 따라 채점한다.

In [108]:
import pandas as pd

train = pd.read_csv('laptop_train.csv')
test = pd.read_csv('laptop_test.csv')

## EDA
train.info()
train.isnull().sum()
test.isnull().sum()

train = train.drop('Model', axis=1)
test = test.drop('Model', axis=1)
train = train.drop('Series', axis=1)
test = test.drop('Series', axis=1)

print ('------ 카테고리 동일 여부 ------')
cols_obj = train.select_dtypes(include='O').columns
for col in cols_obj :
    train_set = set(train[col])
    test_set = set(test[col])
    if train_set == test_set :
        print(col, '동일')
    else :
        print(col, '동일하지 않음')
        
# train.describe()    
# train.describe(include='O')

## 데이터 전처리
y_train = train.pop('Price')

# 결측값 처리
train[cols_obj] = train[cols_obj].fillna('X')
test[cols_obj] = test[cols_obj].fillna('X')

train['RAM'] = train['RAM'].fillna(train['RAM'].mode()[0])
test['RAM'] = test['RAM'].fillna(test['RAM'].mode()[0])

# 원-핫 인코딩
# train_oh = pd.get_dummies(train)
# test_oh = pd.get_dummies(test)
# train.shape, train_oh.shape, test.shape, test_oh.shape
df = pd.concat([train, test])
df_oh = pd.get_dummies(df)
train = df_oh[:len(train)]
test = df_oh[len(train):]
# train.shape, train_oh.shape, test.shape, test_oh.shape

# 스케일링
from sklearn.preprocessing import StandardScaler

cols_num = train.select_dtypes(exclude='O').columns
scaler = StandardScaler()
train[cols_num] = scaler.fit_transform(train[cols_num])
test[cols_num] = scaler.transform(test[cols_num])


## 검증 데이터 나누기
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(train, y_train, test_size=0.2, random_state=0)

## 모델 학습 및 평가
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

rf = RandomForestRegressor(random_state=0)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_val)

r2 = r2_score(y_val, y_pred)
print ('\n------ R² SCORE ------')
print('R²:', r2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91 entries, 0 to 90
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Brand               91 non-null     object 
 1   Model               82 non-null     object 
 2   Series              55 non-null     object 
 3   Processor           86 non-null     object 
 4   Processor_Gen       86 non-null     object 
 5   RAM                 85 non-null     float64
 6   Hard_Disk_Capacity  85 non-null     object 
 7   OS                  85 non-null     object 
 8   Rating              91 non-null     float64
 9   Price               91 non-null     int64  
dtypes: float64(2), int64(1), object(7)
memory usage: 7.2+ KB
------ 카테고리 동일 여부 ------
Brand 동일하지 않음
Processor 동일하지 않음
Processor_Gen 동일하지 않음
Hard_Disk_Capacity 동일하지 않음
OS 동일하지 않음

------ R² SCORE ------
R²: 0.8059084005677305


C:\Users\joajo\AppData\Local\Temp\ipykernel_4168\2304860439.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train[cols_num] = scaler.fit_transform(train[cols_num])
C:\Users\joajo\AppData\Local\Temp\ipykernel_4168\2304860439.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test[cols_num] = scaler.transform(test[cols_num])


#### Q3. 노트북 가격을 예측하시오.
- 제공된 데이터 : `car_train.csv`, `car_test.csv`
- 예측할 컬럼 : `Price`


학습용 데이터(train)을 이용해 중고차 가격을 예측하는 모델을 만든 후 이를 평가용 데이터(test)에 적용해 얻은 예측값을 다음과 같은 형식의 csv 파일로 생성하시오.


제출 파일은 다음 1개의 컬럼을 포함해야 한다.
- pred : 예측값 (가격)
- 제출 파일명 : `new9.csv`


제출한 모델의 성능은 RMSLE 평가지표에 따라 채점한다.

In [32]:
import pandas as pd

train = pd.read_csv('car_train.csv')
test = pd.read_csv('car_test.csv')

## EDA
# train.info()
# train.isnull().sum()
# test.isnull().sum()
# train.describe()
# train.describe(include='O')
# train.shape, test.shape # ((6732, 17), (5772, 16))

print ('------ 카테고리 동일 여부 ------')
cols_obj = train.select_dtypes(include='O').columns
for col in cols_obj :
    train_set = set(train[col])
    test_set = set(test[col])
    if train_set == test_set :
        print(col, '동일')
    else :
        print(col, '동일하지 않음')
        
## 데이터 전처리
y_train = train.pop('Price')

train['Turbo'] = train['Engine volume'].str.contains('Turbo').astype(int)
test['Turbo'] = test['Engine volume'].str.contains('Turbo').astype(int)

train['Engine volume'] = train['Engine volume'].str.replace('Turbo', '').astype(float)
test['Engine volume'] = test['Engine volume'].str.replace('Turbo', '').astype(float)

train['Mileage'] = train['Mileage'].str.split().str[0].astype(int)
test['Mileage'] = test['Mileage'].str.split().str[0].astype(int)

# # 스케일링
# from sklearn.preprocessing import RobustScaler

# cols_num = train.select_dtypes(exclude='O').columns
# scaler = RobustScaler()
# train[cols_num] = scaler.fit_transform(train[cols_num])
# test[cols_num] = scaler.transform(test[cols_num])

# 원-핫 인코딩
# df = pd.concat([train, test])
# df_oh = pd.get_dummies(df)
# train_oh = df_oh.iloc[:len(train)]
# test_oh = df_oh.iloc[len(train):]
# print(train.shape, train_oh.shape, test.shape, test_oh.shape)

# 라벨 인코딩
from sklearn.preprocessing import LabelEncoder

df = pd.concat([train, test])
for col in cols_obj :
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    
train = df.iloc[:len(train)]
test = df.iloc[len(train):]
    
# print(train.shape, test.shape)

## 검증 데이터 나누기
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(train, y_train, test_size=0.2, random_state=0)

## 모델 학습 및 평가
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import root_mean_squared_log_error

rf = RandomForestRegressor(random_state=0, n_estimators=200)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_val)

rmsle = root_mean_squared_log_error(y_val, y_pred)
print ('\n------ RMSLE SCORE ------')
print('RMSLE:', rmsle)

# 예측 및 결과 파일 생성
pred = rf.predict(test)
result = pd.DataFrame({'pred':pred}).to_csv('new9.csv', index=False)
pd.read_csv('new9.csv').head()

------ 카테고리 동일 여부 ------
Levy 동일하지 않음
Manufacturer 동일하지 않음
Model 동일하지 않음
Category 동일
Leather interior 동일
Fuel type 동일하지 않음
Engine volume 동일하지 않음
Mileage 동일하지 않음
Gear box type 동일
Drive wheels 동일
Doors 동일
Wheel 동일
Color 동일

------ RMSLE SCORE ------
RMSLE: 1.0822004244243006


,pred
0,40019.835
1,10820.880
2,16399.925
3,69105.440
4,47603.045
